<a href="https://colab.research.google.com/github/sarthakksingh2/CAC_SummerAnalyticsFinal/blob/main/CAC_IITG_CapstoneP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pathway bokeh --quiet

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [4]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("dataset.csv")

# Basic Info
print(df.head())
print(df.info())
print(df.describe())


   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime  
0        07:59:00  
1        08:25:00  
2        08:59:00  
3     

In [5]:
# Feature Engineering
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']

# Normalize key features
df['norm_queue'] = df['QueueLength'] / df['QueueLength'].max()
df['norm_occupancy'] = df['Occupancy'] / df['Occupancy'].max()
df['norm_capacity'] = df['Capacity'] / df['Capacity'].max()


# Vehicle type weight encoding (adjustable)
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_weights)

# Encode special day as binary
df['IsSpecialDay'] = df['IsSpecialDay'].astype(int)

# Set base price
df['price'] = 10.0


In [6]:
def model1_baseline_linear(df, alpha=2.0):
    df = df.copy()
    df['price_next'] = df['price'] + alpha * df['occupancy_rate']
    return df[['timestamp', 'parking_lot_id', 'price', 'price_next']]


In [7]:
# Combining the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [10]:
# Save the selected columns to a CSV file for streaming
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

In [11]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity


In [12]:
# Load the data as a simulated stream using Pathway's replay_csv function

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [13]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [14]:
def model2_demand_based(df, base_price=10.0, lam=0.5):
    df = df.copy()

    # Demand function (linear form)
    df['demand'] = ( 2.0 * df['occupancy_rate'] + 1.5 * df['norm_queue'] - 1.0 * df['norm_traffic'] + 2.0 * df['is_special_day'] + 1.2 * df['vehicle_weight']
    )

    # Normalize demand
    df['norm_demand'] = (df['demand'] - df['demand'].min()) / (df['demand'].max() - df['demand'].min())

    # Calculate new price
    df['price_new'] = base_price * (1 + lam * df['norm_demand'])

    # Apply price bounds
    df['price_new'] = df['price_new'].clip(lower=0.5 * base_price, upper=2 * base_price)

    return df[['timestamp', 'parking_lot_id', 'price_new', 'norm_demand']]


In [16]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed
    )
    .with_columns(

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


In [18]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [19]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()